In [290]:
import csv
from random import randint

In [291]:
genderTable={
    "Male":0,
    "Female":1
}
schoolTable={
    "CCDS":1,
    "EEE":2,
    "CoB (NBS)":3,
    "SoH":4,
    "WKW SCI":5,
    "CoE":6,
    "MAE":7,
    "SPMS":8,
    "SBS":9,
    "SSS":10,
    "ASE":11,
    "NIE":12,
    "ADM":13,
    "MSE":14,
    "LKCMedicine":15,
    "CCEB":16,
    "CEE":17,
    "HASS":18
}

In [292]:
class Student:
    gender=""
    school=""
    CGPA=""
    tutorialGroup=""
    ID=""
    name=""
    
    team=""
    def __init__(self,gender,school,CGPA,tutorialGroup,ID,name):
        self.gender=gender
        self.school=school
        self.CGPA=CGPA
        self.tutorialGroup=tutorialGroup
        self.name=name
        self.ID=ID
    def isDigit(self,target):
        return ord(target) in range(ord('0'),ord('9')+1)
    def getCGPA(self):
        res=0
        # if len(self.CGPA)==1:
        #     return 100*int(self.CGPA)
            
        # else:
        #     for i in self.CGPA:
        #         if self.isDigit(i):
        #             res=res*10+ord(i)-ord('0')
        #     return res
        return float(self.CGPA)*100
    def getGender(self):
        return genderTable[self.gender]
    def getSchool(self):
        return schoolTable[self.school]
    def getTutorialGroup(self):
        res=0
        for i in self.tutorialGroup:
            if self.isDigit(i):
                res=res*10+ord(i)-ord('0')
        return res
    def display(self):
        print(self.tutorialGroup,self.school,self.CGPA,self.gender,self.team)

In [293]:
def countAvg(group:list[Student]):
    num=0
    avgGender=0
    avgCGPA=0
    for student in group:
        avgGender=avgGender+student.getGender()
        avgCGPA=avgCGPA+student.getCGPA()
    return (avgGender/10.0),(avgCGPA/10.0)

In [294]:
#Get distance

# def distance(targetGender:float,targetCGPA:float,presentGender:float,presentCGPA:float,candidate:Student,step:int):
#     newGender=(presentGender*step+candidate.getGender())/(step+1)
#     newCGPA=(presentCGPA*step+candidate.getCGPA())/(step+1)
#     distGender=newGender/targetGender-1
#     distCGPA=newCGPA/targetCGPA-1
#     return (distGender**2+distCGPA**2)**0.5,newGender,newCGPA

def distance(targetGender:float,targetCGPA:float,presentList:list[Student],coefficientGender=1,coefficientCGPA=1,coefficientSchool=1):
    presentGender=0
    presentCGPA=0
    schoolRecord=[0 for i in range(0,19)]
    maxSame=0
    for student in presentList:
        schoolRecord[student.getSchool()]+=1
        presentCGPA+=student.getCGPA()
        presentGender+=student.getGender()
    for school in schoolRecord:
        maxSame=max(maxSame,school)
    distGender=(1/(2.71828*coefficientGender))*(((presentGender/5.0)-targetGender))*2.71828**(coefficientGender*((presentGender/5.0)-targetGender)-coefficientGender/2.71828)
    distCGPA=((presentCGPA/5.0)-targetCGPA)*2.71828**((presentCGPA/5.0)-targetCGPA-1)
    distSchool=(1/40.0)*(maxSame-1)*2.71828**maxSame

    return coefficientGender*abs(distGender)**2+coefficientCGPA*distCGPA%1**2+coefficientSchool*(distSchool)**2

def distanceGroup(groupList:list[Student],coefficientGender=1,coefficientCGPA=1,coefficientSchool=1):
    targetGender=0
    targetCGPA=0
    for student in groupList:
        targetGender+=student.getGender()
        targetCGPA+=student.getCGPA()
    targetGender/=50.0
    targetCGPA/=50.0
    groupDistance=0
    for i in range(0,10):
        teamList:Student=[]
        for j in range(5*i,5*i+5):
            teamList.append(groupList[j])
        groupDistance+=distance(targetGender,targetCGPA,teamList,coefficientGender,coefficientCGPA,coefficientSchool)
    return groupDistance


In [295]:
#Read file
studentList:list[list[Student]]=[[] for i in range(0,121)]
with open("records.csv","r",encoding="utf-8") as records:
    reader=csv.reader(records)
    flag=True
    for row in reader:
        if flag:
            flag=False
            continue
        present=Student(row[4],row[2],row[5],row[0],row[3],row[1])
        # print(row)
        # print(present.getTutorialGroup())
        # present.display()
        studentList[present.getTutorialGroup()].append(present)
# for tutorialGroup in studentList:
#     for student in tutorialGroup:
#         student.display()

In [296]:
# i=0
# for tutorialGroup in studentList:
#     if i==0:
#         i+=1
#         continue
#     print(tutorialGroup[0].getTutorialGroup(),distanceGroup(tutorialGroup))
#     i+=1

In [297]:
teamID=1
flag=True
for tutorialGroup in studentList:
    if flag:
        flag=False
        continue
    #Part by Li Yitong
    T=10000.0
    f=0.99
    while T>0.001:
        p1=0
        p2=0
        while p1%5==p2%5:
            p1=randint(0,49)
            p2=randint(0,49)
        pre=-distanceGroup(tutorialGroup,4,1,0.5)
        #swap
        tutorialGroup[p1],tutorialGroup[p2]=tutorialGroup[p2],tutorialGroup[p1]

        aft=-distanceGroup(tutorialGroup,4,1,0.5)
        if pre>aft:
            print((aft-pre)/T)
        if pre>aft and randint(0,1)<1-(2.71828**((aft-pre)/T)):
            # swap(tutorialGroup[p1],tutorialGroup[p2])
            tutorialGroup[p1],tutorialGroup[p2]=tutorialGroup[p2],tutorialGroup[p1]
        T*=f
    for i in range(0,10):
        for j in range(5*i,5*i+5):
            tutorialGroup[j].team=teamID
        teamID+=1
    print()
i=0
for tutorialGroup in studentList:
    for student in tutorialGroup:
        student.display()
        i+=1
        if i%5==0:
            print()

with open("output.csv","w",encoding="utf-8") as output:
    writer=csv.writer(output)
    writer.writerow(["Tutorial Group","Student ID","School","Name","Gender","CGPA"])
    for tutorialGroup in studentList:
        for student in tutorialGroup:
            writer.writerow([student.tutorialGroup,student.ID,student.school,student.name,student.gender,student.CGPA])
# for tutorialGroup in studentList:
#     print(len(tutorialGroup))

-8.712207314423909e-05
-7.257434760782057e-05
-9.393958648643638e-05
-1.4939313629588522e-05
-5.2150662793545044e-05
-4.416747358409181e-05
-2.2297328515898698e-06
-8.110366592925515e-05
-7.792136246532372e-05
-4.243924689682503e-06
-4.308140177250395e-07
-1.4752995817686529e-05
-0.00020380042332550079
-3.0115751497895968e-05
-7.267197231181014e-05
-0.00010865492003255182
-0.00011246397112726392
-1.2397093008427491e-05
-0.00010278566843808555
-9.406815181679494e-05
-1.7965906695835696e-07
-0.00013083789416906092
-9.427759656829115e-05
-3.7900460247257345e-05
-8.800177635135527e-05
-5.381627304445723e-06
-7.740905910580965e-05
-4.3881676414826395e-05
-0.0001253535496961427
-0.0001323084341784109
-7.983152162947122e-06
-5.429099184235939e-05
-9.040319596358452e-05
-9.928415714190469e-05
-8.180071952511387e-05
-2.4595472102634188e-05
-0.00010278008023845576
-0.00016461282681738554
-0.00016698280979620653
-4.2748454683010736e-05
-6.167588436874422e-05
-0.00019664699817805912
-0.00015805971